In [1]:
#installing libraries
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
#dowloading vader lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sarah\AppData\Roaming\nltk_data...


True

In [6]:
#creating a functios to fetch data from a sql database
def fetch_data_from_sql():
    conn_str =(
        "Driver={SQL Server};"
        "Server=SARAH\\SQLEXPRESS;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    conn = pyodbc.connect(conn_str)

    querry = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"

    df = pd.read_sql(querry, conn)

    conn.close()
    return df


In [7]:
customer_reviews_df = fetch_data_from_sql()

C:\Users\sarah\AppData\Local\Temp\ipykernel_19392\1946123401.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(querry, conn)


In [8]:
customer_reviews_df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."
...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special."
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.


In [9]:
#initializing the vader sentiment analyzer
sia = SentimentIntensityAnalyzer()

In [10]:
#defining a function to calculate the sentiment analyzes
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    #return the compound score: -1 (most negative) and 1 (most positive)
    return sentiment['compound']

In [11]:
#defining a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    if score > 0.05: #Positive sentiment score
        if rating >= 4:
            return 'Positive' #high rating and Positive sentiment
        elif rating == 3:
            return 'Mixed Positive' #Neutral rating but positive sentiment
        else:
            return 'Mixed Negative' #Low rating but positive sentiment
    elif score < -0.05: #Negative sentiment score
        if rating <= 2:
            return 'Negative' # low rating and negative sentiment
        elif rating == 3: 
            return 'Mixed Negative' #Neutral rating but positive sentiment
        else:
            return 'Mixed Positive' # high rating but negative sentiment
    else: #neutral sentiment score
        if rating >= 4:
            return 'Positive' #high rating with neutral sentiment
        elif rating <= 2:
            return 'Negative' #Low rating with neutral sentiment
        else:
            return 'Neutral' #neutral rating and neutral sentiment



In [12]:
#creating a function to bucket sentiment scores into text ranges

def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0' #strong positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49' #mid positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0' # mid negative sentiment
    else:
        return '-1.0 to -0.5' #strong negative sentiment

In [13]:
#applying the sentiment analysis to calculate sentiment scores for each review
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

In [14]:
#applying the sentiment categorization using text and rating
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1
)

In [15]:
#applying sentiment bucketing to categorize scores into defined ranges
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

In [16]:
print(customer_reviews_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [18]:
#Saving the dataframe
customer_reviews_df.to_csv('customer_reviews_with_sentiment.csv', index= False)